<a href="https://colab.research.google.com/github/mmasliakova/Ribo-Seq-analysis/blob/main/Differential_expression_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

In [ ]:
BiocManager::install("DESeq2")

In [ ]:
library(DESeq2)

In [4]:
coldata <- read.table('design.tsv', header = TRUE, sep = '\t')

In [5]:
head(coldata)

,X,condition,experiment
,<chr>,<chr>,<chr>
1,LC001-normal-RPF,normal,RPF
2,LC001-tumor-RPF,tumor,RPF
3,LC001-normal-RNA,normal,RNA
4,LC001-tumor-RNA,tumor,RNA
5,LC033-normal-RPF,normal,RPF
6,LC033-tumor-RPF,tumor,RPF


In [6]:
rpf_data = coldata[coldata$experiment == 'RPF', ]
rpf_data = rpf_data[, -3]

In [7]:
rpf_data <- data.frame(rpf_data[,-1], row.names = rpf_data[,1])

In [8]:
colnames(rpf_data) = c('condition')

In [9]:
rna_data = coldata[coldata$experiment == 'RNA', ]
rna_data = rna_data[, -3]

In [10]:
rna_data <- data.frame(rna_data[,-1], row.names = rna_data[,1])
colnames(rna_data) = c('condition')

In [11]:
cts = read.table('01. RiboSeq_RNASeq_HCC_counts.tsv', header = TRUE, sep = '\t')
cts[is.na(cts)] <- 0
cts = cts[, -1:-2]
cts$LC509.tumor.RNA = as.integer(cts$LC509.tumor.RNA)
names(cts) <- gsub("\\.", "\\-", names(cts))

In [12]:
head(cts)

,LC001-normal-RPF,LC001-tumor-RPF,LC001-normal-RNA,LC001-tumor-RNA,LC033-normal-RPF,LC033-tumor-RPF,LC033-normal-RNA,LC033-tumor-RNA,LC034-normal-RPF,LC034-tumor-RPF,⋯,LC507-normal-RNA,LC507-tumor-RNA,LC508-normal-RPF,LC508-tumor-RPF,LC508-normal-RNA,LC508-tumor-RNA,LC509-normal-RPF,LC509-tumor-RPF,LC509-normal-RNA,LC509-tumor-RNA
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,0,402,873,0,0,301,1627,0,0,⋯,218,457,0,0,200,889,0,0,161,857
2,0,0,0,1,0,0,0,1,0,0,⋯,0,0,1,0,0,0,0,0,0,0
3,168,239,88,187,114,439,66,201,325,117,⋯,92,170,109,357,60,244,141,173,56,83
4,65,121,60,106,36,163,70,156,130,76,⋯,127,176,67,176,55,261,64,100,68,176
5,8,43,18,70,16,76,4,63,44,14,⋯,124,437,7,167,34,201,6,61,9,58
6,0,6,19,9,14,13,61,24,15,62,⋯,94,13,3,9,42,26,7,10,26,57


In [13]:
rpf_cts = cts[, grepl('RPF', names(cts))]

In [14]:
head(rpf_cts)

,LC001-normal-RPF,LC001-tumor-RPF,LC033-normal-RPF,LC033-tumor-RPF,LC034-normal-RPF,LC034-tumor-RPF,LC501-normal-RPF,LC501-tumor-RPF,LC502-normal-RPF,LC502-tumor-RPF,LC505-normal-RPF,LC505-tumor-RPF,LC506-normal-RPF,LC506-tumor-RPF,LC507-normal-RPF,LC507-tumor-RPF,LC508-normal-RPF,LC508-tumor-RPF,LC509-normal-RPF,LC509-tumor-RPF
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0
3,168,239,114,439,325,117,106,192,118,292,159,237,149,230,169,260,109,357,141,173
4,65,121,36,163,130,76,34,43,84,221,74,185,65,98,81,105,67,176,64,100
5,8,43,16,76,44,14,5,27,24,42,10,71,18,103,79,225,7,167,6,61
6,0,6,14,13,15,62,5,5,22,5,0,4,18,6,8,1,3,9,7,10


In [15]:
rna_cts = cts[, grepl('RNA', names(cts))]

RPF

In [16]:
rpf_data$condition = factor(rpf_data$condition)
dds <- DESeqDataSetFromMatrix(countData = rpf_cts,
                              colData = rpf_data,
                              design = ~ condition)

In [ ]:
dds = DESeq(dds)

In [18]:
res = results(dds)
rpf_res = as.data.frame.matrix(res)

In [19]:
write.table(rpf_res, file = 'rpf_deseq.tsv', quote = FALSE, sep = '\t', col.names = NA)

RNA

In [20]:
rna_data$condition = factor(rna_data$condition)
dds <- DESeqDataSetFromMatrix(countData = rna_cts,
                              colData = rna_data,
                              design = ~ condition)

In [ ]:
dds = DESeq(dds)

In [22]:
res = results(dds)
rna_res = as.data.frame.matrix(res)

In [23]:
write.table(rna_res, file = 'rna_deseq.tsv', quote = FALSE, sep = '\t', col.names = NA)

BOTH

In [24]:
coldata <- data.frame(coldata[,-1], row.names = coldata[,1])

In [25]:
coldata$condition = factor(coldata$condition)
coldata$experiment = factor(coldata$experiment)
dds <- DESeqDataSetFromMatrix(countData = cts,
                              colData = coldata,
                              design = ~ experiment + condition + condition:experiment)

In [ ]:
dds = DESeq(dds)

In [27]:
res = results(dds)
res = as.data.frame.matrix(res)

In [28]:
write.table(res, file = 'total_deseq.tsv', quote = FALSE, sep = '\t', col.names = NA)